# **Chains in LangChain**

## **Outline**

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers.

In [3]:
# ! pip install langchain langchain_groq

### **LLMChain**

For using `ChatGroq`, we first need to set API Key.

In [4]:
import os

os.environ["GROQ_API_KEY"] = "YOUR-API-KEY"

`LLMChain` is a class that facilitates the creation of chains that combine language models with other functionalities.

It simplifies the process of setting up a workflow where an LLM is used to process input and generate output, possibly in conjunction with other steps.

In [5]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


## Create the langchain model:
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.9
)

## **Key Components of LLMChain**

- **LLM:**
This is the language model you are using. It can be any model that conforms to the LangChain’s interface for language models.

- **Prompt:** This defines how the input should be formatted and what specific instructions or context should be given to the language model.

- **Output Parsing:** This handles the parsing of the output from the language model into a format that can be used for subsequent steps or final output.

In [6]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Create the chain:
chain = LLMChain(llm = llm,
                 prompt = prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
from IPython.display import display, Markdown

product = "Hats with shape of animation characters."

# Invoke the chain
display(Markdown(chain.run(product)))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


What a fun question!

Here are some name suggestions for a company that makes hats with shapes of animation characters:

1. **ToonTopia**: A playful combination of "toon" (short for cartoon) and "utopia," implying a fantastical world of animated characters on hats.
2. **Character Caps**: Simple and straightforward, emphasizing the focus on character-shaped hats.
3. **Hatimation**: A blend of "hat" and "animation," highlighting the unique fusion of fashion and animation.
4. **Pixie Peaks**: "Pixie" evokes a sense of playful, whimsical designs, while "Peaks" references the hat shapes.
5. **Fable Factory**: "Fable" nods to the fantastical stories and characters, while "Factory" implies a creative, industrious approach to hat-making.
6. **Cartoon Couture**: This name combines the world of cartoons with high-fashion "couture," suggesting stylish, character-inspired hats.
7. **Animate Apparel**: This name emphasizes the animated characters while also highlighting the clothing aspect of the hats.
8. **Storybook Style**: This name taps into the nostalgic, storytelling aspect of animation, implying hats that evoke a sense of wonder and imagination.
9. **Charm Caps**: "Charm" conveys the delightful, whimsical nature of the hats, while "Caps" is a colloquialism for hats.
10. **Fantasy Fusion**: This name highlights the blending of fantasy worlds (animation) with functional fashion (hats).

Pick the one that resonates with your brand's personality and style!

## **SimpleSequentialChain**

- The simplest form of a sequential chain is where each step has a single input and output.

- The output of one step is passed as input to the next step in the chain. You would use `SimpleSequentialChain` when you have a **linear pipeline** where each step has a single input and output, it implicitly passes the output of one step as input to the next.

- This is great for composing a precise sequence of LLMChains where each builds directly on the previous output.

In [8]:
from langchain.chains import SimpleSequentialChain

## Create the llm model:
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.8
)

## Create the prompt template:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give just one name."
)

## Create the chain:
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt
)

Now, we will create the second chain:

In [9]:
## Template for the second prompt
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following, make sure to use the company name in this description. \
    company:{company_name}"
)

## Create the second chain:
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt
)

In the next step, we use the `SimpleSequentialChain` to connect those two chains:

### **What is SimpleSequentialChain?**
`SimpleSequentialChain` helps in chaining together multiple `LLMChain` instances or other chains in a linear sequence.

This allows you to build complex workflows where the data flows through each step in a defined order.

In [10]:
overall_simple_chain = SimpleSequentialChain(
    chains = [chain_one, chain_two],
    verbose = True
)

In [11]:
print(f"Product is: \"{product}\"")

Product is: "Hats with shape of animation characters."


In [12]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
What a unique and fun idea!

Here's a suggestion for a company name that might fit the bill:

**ToonTops**

This name plays off the idea of "toons" (short for cartoons or animations) and "tops" (referring to hats being worn on top of the head). It's catchy, easy to remember, and immediately conveys the theme of the company's products.
ToonTops brings whimsy to wearables with customizable, interchangeable hat toppers featuring beloved cartoon characters and quirky designs.

> Finished chain.


'ToonTops brings whimsy to wearables with customizable, interchangeable hat toppers featuring beloved cartoon characters and quirky designs.'

## **SequentialChain**

- A more general form of sequential chain allows multiple inputs and outputs per step.

- You would use `SequentialChain` when you have a more complex pipeline where steps might have multiple inputs and outputs.

- `SequentialChain` allows you to explicitly specify all the input and output variables at each step and map outputs from one step to inputs of the next. This provides more flexibility when steps might have multiple dependencies or produce multiple results to pass along.

In [13]:
from langchain.chains import SequentialChain

llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.9
)

## prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

## chain1: Review ---> Translate into English
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    output_key = "English_Review"
)

In [14]:
## prompt template 2: summarize the english text
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain2: English Review ---> Summarize
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt,
    output_key = "summary"
)

In [15]:
# prompt template 3: Find the language of the initial review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain3: Review ---> language
chain_three = LLMChain(
    llm = llm,
    prompt = third_prompt,
    output_key = "language"
)

In [16]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary just in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain4: summary, language ---> Response
chain_four = LLMChain(llm = llm,
                      prompt = fourth_prompt,
                      output_key = "followup_message"
                     )

In [17]:
# overall_chain: Review ---> English_Review, summary, language, followup_message

overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables = ["English_Review", "summary", "language", "followup_message"],
    verbose = True
)

In [18]:
review = "Creo que es un producto maravilloso, su precio es un poco elevado, pero creo que merece la pena."

overall_chain(review)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Creo que es un producto maravilloso, su precio es un poco elevado, pero creo que merece la pena.',
 'English_Review': 'Here is the translation:\n\n"I think it\'s a wonderful product, the price is a bit high, but I think it\'s worth it."',
 'summary': 'Here is a summary of the review in 1 sentence:\n\nThe reviewer thinks the product is wonderful and worth the high price.',
 'language': 'The language of the review is Spanish.\n\nHere\'s a translation of the review:\n\n"I think it\'s a wonderful product, its price is a bit high, but I think it\'s worth it."',
 'followup_message': 'Excelente elección! El producto es verdaderamente increíble y la inversión vale la pena.'}

## **When to use `SequentialChain`**

- When your workflow requires more complex logic, such as conditionally executing steps based on intermediate results.
- When you need to manipulate the inputs and outputs between steps in more detailed ways.
- For workflows that involve branching, looping, or more advanced error handling.

## **Router Chain**

- Router chains allow routing inputs to different destination chains based on the input text. This allows the building of chatbots and assistants that can handle diverse requests.

  - Router chains examine the input text and route it to the appropriate
  destination chain.
  - Destination chains handle the actual execution based on the input.
  - Router chains are powerful for building multi-purpose chatbots/assistants.


In [19]:
## Physics & Math Template:

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

In [20]:
## History & ComputerScience Template:

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

Now, we merge these four templates into a list of dictionaries that each dictionary has three components:
- `name`
- `description`
- `prompt_template`

We will use the `name` and `prompt_template` to create

In [45]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },

    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },

    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },

    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

Now that we have defined different routers prompt templates, we create the llm

## **MultiPromptChain:**

The `MultiPromptChain` works by defining multiple prompts and a mechanism to decide which prompt to use for a given input. It uses a router to make this decision, and each prompt can be tailored to handle a specific type of query or task.
It takes multiple arguments as its inputs:

- **`router_chain`**: The `router_chain` is a specialized chain responsible for determining which sub-chain or prompt should handle the input. It is essentially the decision-making component.

- **`destination_chains`**: These are the actual sub-chains or pipelines that handle specific types of inputs. Each destination_chain corresponds to a particular task or type of query.

- **`default_chain`**: The `default_chain` is a fallback or catch-all chain that handles any inputs that do not match the criteria for the destination_chains.

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

## LLM
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.1
)

So first, we want to create the `router_chain`, for that, we use the **`LLMRouterChain`**.

In [47]:
## First, we need to store all different routs with their names and chains into a dictionary.
destination_chains = {}

for p_info in prompt_infos:

    ## Extract the name and template
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]

    ## Convert the raw text template into a prompt template
    prompt = ChatPromptTemplate.from_template(template = prompt_template)

    ## Create the chain based on the prompt
    chain = LLMChain(llm = llm,
                     prompt = prompt)

    ## Add the chain with its corresponding name to dictionary
    destination_chains[name] = chain

In [48]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

We need to feel the `destination` in our prompt, in consist of a name and the description corresponds to that name.

In [49]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [50]:
## Fill the destination
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)


## Create the prompt
router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser()
)


## Create the chain
router_chain = LLMRouterChain.from_llm(
    llm = llm,
    prompt = router_prompt
)

In [51]:
## Checkout the final router prompt:
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics: Good for answering questions about physics\nmath: Good for answering math q

The next tep is to create the default chain, A fallback chain for inputs that do not match any specific criteria.

In [60]:
default_prompt = ChatPromptTemplate.from_template("Say 'HELLO LEONARDO' at the begging and then answer the question that has been mentioned in the following 'input': {input}")

default_chain = LLMChain(
    llm = llm,
    prompt = default_prompt
)

# **FINALLY**

In [61]:
## Connect the router, destination and default chain to eachother:
chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = destination_chains,
    default_chain = default_chain,
    verbose = True
)

In [62]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is blackbody radiation?'}
> Finished chain.


"Blackbody radiation is a fundamental concept in physics, and I'm happy to explain it in simple terms.\n\nBlackbody radiation is the thermal radiation emitted by an idealized perfect absorber of electromagnetic radiation, known as a blackbody. In other words, it's the radiation emitted by an object that absorbs all the electromagnetic radiation that falls on it, rather than reflecting or transmitting any of it.\n\nWhen an object is heated, its atoms or molecules gain energy and start vibrating rapidly. As they vibrate, they emit electromagnetic radiation across a wide range of frequencies, including visible light, infrared, and even X-rays. The distribution of this radiation is characteristic of the object's temperature.\n\nThe key feature of blackbody radiation is that it follows a specific spectrum, known as Planck's law, which describes the energy distribution of the radiation as a function of frequency or wavelength. This spectrum is universal, meaning it's the same for all objects

In [63]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is the value of 2 + 2'}
> Finished chain.


"A classic!\n\nTo answer this question, I'll break it down into its simplest component parts:\n\n**Component Part 1:** 2\n**Component Part 2:** 2\n\nNow, I'll answer each component part individually:\n\n**Component Part 1:** 2 = 2 (no calculation needed, it's just 2!)\n\n**Component Part 2:** 2 = 2 (again, no calculation needed, it's just 2!)\n\nNow, I'll combine the answers to the component parts to get the final answer:\n\n**Final Answer:** 2 + 2 = 2 + 2 = **4**\n\nSo, the value of 2 + 2 is **4**!"

### **Default Chain Selection:**

**Fallback Mechanism:** If the input does not match any of the criteria defined for the specific destination chains, the router chain will fall back to the `default_chain.`


**Uncertainty Handling:** In some implementations, if the language model is unsure about the best route or if the confidence score for routing to a specific chain is below a certain threshold, it may also route the input to the `default_chain.`

In [64]:
chain.run("How old is lionel messi?")



> Entering new MultiPromptChain chain...
None: {'input': 'Who is Lionel Messi and how old is he?'}
> Finished chain.


"HELLO LEONARDO!\n\nLionel Messi is a professional soccer player who is widely regarded as one of the greatest players of all time. He was born on June 24, 1987, which makes him 35 years old as of June 2022. Messi has played for FC Barcelona and the Argentina national team, winning numerous accolades including six Ballon d'Or awards, ten La Liga titles, and four UEFA Champions League titles. He is known for his exceptional dribbling skills, speed, and goal-scoring ability."